In [1]:
import json
#import pydevd
#pydevd.settrace('localhost', port=49309, stdoutToServer=True, stderrToServer=True)
import tensorflow as tf

import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import OrderedDict, defaultdict
from six.moves import xrange
#from __pynauty__ import graph
#import pynauty.graph
import pynauty
import time
import tensorflow as tf
import networkx as nx
import numpy as np
import pynauty as nauty
from multiprocessing import Pool
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from PatchyTools import Dataset
from PatchyTools import utils
from PatchyTools import Patchy_san

In [2]:
from sklearn.model_selection import train_test_split

### Functions

In [3]:
mutag = Dataset.Dropbox('MUTAG')
df_edge_label = mutag.get_edge_label()
df_graph_ind = mutag.get_graph_ind()
df_adj = mutag.get_adj()
df_node_label = mutag.get_node_label()
df_node_label = pd.concat([df_node_label, df_graph_ind.graph_ind], axis=1)
del df_graph_ind

num_features = len(df_node_label.label.unique())

data_in_patchysan = Patchy_san.main(WIDTH_W=18, RECEPTIVE_FIELD_SIZE_K=10, datasetname='MUTAG')

final_labels = 1+mutag.get_graph_label().graph_label.values

# Logistic Regression

In [5]:

from sklearn.linear_model import SGDClassifier
ret =[]

for seed in range(10):
    X_train, X_test, y_train, y_test = train_test_split(data_in_patchysan, final_labels, 
                                                        test_size=0.10, random_state=seed)
    lr_model = SGDClassifier(loss="log", penalty="l2", max_iter=1000,tol=1e-3 )
    X_train = X_train.reshape( (X_train.shape[0],X_train.shape[1]*X_train.shape[2]*X_train.shape[3]))
    lr_model.fit(X_train,y_train)
    X_test = X_test.reshape( (X_test.shape[0],X_test.shape[1]*X_test.shape[2]*X_test.shape[3]))
    y_pred = lr_model.predict(X_test)
    ret.append(accuracy_score(y_test, y_pred))
    
    
print (np.mean(ret), np.std(ret))

0.868421052632 0.0676064872561
